In [ ]:
#Import Libraries
import numpy as np
from queue import PriorityQueue
from cv2 import VideoWriter, VideoWriter_fourcc 
import Node



def line(point1, point2, p, q, r):
    
    a=(point2[1] - point1[1])
    b=p - point1[0]
    c=point2[0] - point1[0]
    d=point1[1] + r - q
    distance = ( a*b ) / ( c) + (d)
    
    return distance
#following function creates a map

def create_map():
    map = np.zeros((250,600)) # height: 250; width: 600
    
    radius = 0                                       
    clearance = 5                                       
    t = radius + clearance                                   # Total clearance
    
   
   
    for i in range(map.shape[1]):
        for j in range(map.shape[0]):
            #triangle 
            
           if (i>(460-t)and i<(510+t)and j>(25-t) and j< (225+2*t) and line((460,225),(510,125),i,j,t) > 0
               and line((460,25),(510,125),i,j,t) < 0):
                map[j,i]=1
            # Lower rectangle

           if (i > (100-t) and i < (150+t) and line((100,150),(150,150),i,j,t) < 0 
                and line((100,250),(150,250),i,j,t) > 0):
                map[j,i]=1
                
            #Upper rectangle
            
           if (i > (100-t) and i < (150+t) and line((100,0),(150,0),i,j,t) < 0 
                and line((100,100),(150,100),i,j,t) > 0):
                map[j,i]=1
                
            # Hexagon
            
           if (i > (235.05-t) and i < (364.95+t) and line((235.05,87.5),(300,50),i,j,t) < 0 
                and line((300,50),(364.95,87.5),i,j,t) < 0 
                and line((235.05,162.5),(300,200),i,j,t) > 0 
                and line((300,200),(364.95,162.5),i,j,t) > 0):
                map[j,i]=1
            
         
           if (i > 0 and i < t):
                map[j,i] = 1
           if (i < 600 and i > (600-t)):
                map[j,i] = 1
           if (j > 0 and j < t):
                map[j][i] = 1
           if (j < 250 and j >(250 - t)):
                map[j][i] = 1    
    return map

def collision_check(point,map):
    flag = False
    
    if map[point[1],point[0]] == 1:
        flag = True
    
    return flag


def goal(map):
    
    flag = False
    while not flag:
        goal_node = [int(item) for item in input("Enter target node(x,y): ").split(',')]
        goal_node[1] = 250 - goal_node[1]
        if (len(goal_node) == 2 and (0 <= goal_node[0] <= 600) and (0 <= goal_node[1] <= 250)):
            if not collision_check(goal_node,map):
                flag = True
            else:
                print("obstacle collision \n")
        else:
            print("goal node invalid.\n")
            flag = False
        
    return goal_node

def neighbours(node,map):

    x = node.x
    y = node.y
    # Actions Set = {(1,0), (-1,0), (0,1), (0,-1), (1,1), (-1,1), (1,-1), (-1,-1)} 
    m = [(x, y + 1),(x + 1, y),(x, y -1),(x - 1, y),(x + 1, y + 1),(x + 1, y - 1),(x - 1, y - 1),(x - 1, y + 1)]
    v_paths = []
    for pos, m in enumerate(m):
        if not (m[0] >= 600 or m[0] < 0 or m[1] >= 250 or m[1] < 0):
            if map[m[1]][m[0]] == 0:
                cost = 1.4 if pos > 3 else 1
                v_paths.append([m,cost])

    return v_paths

def Dijkstra_algorithm(goal_node, map):
 
    flag = False 
    while not flag:
        start_node = [int(item) for item in input("\n Enter starting node(x,y): ").split(',')]
        start_node[1] = 250 - start_node[1]

        if (len(start_node) == 2 and (0 <= start_node[0] <= 600) and (0 <= start_node[1] <= 250)):
            if not collision_check(start_node,map):
                flag = True
            else:   
                print("obstacle collision \n")
        else:
            print("start node invalid.\n")
            flag = False
            
    print(" Finding path\n")

    q = PriorityQueue()                                                              
    visited = set([])                                                              
    node_objects = {}                                                              
    distance = {}                                                                     
    
    distance = {(i, j): float('inf') for i in range(map.shape[1]) for j in range(map.shape[0])}

    distance[str(start_node)] = 0                                                   
    visited.add(str(start_node))                                                     
    node = Node.Node(start_node,0,None)                                              
    node_objects[str(node.pos)] = node                                              
    q.put([node.cost, node.pos])                                                    

    while not q.empty():                                                             
        node_temp = q.get()                                                          
        node = node_objects[str(node_temp[1])]  
  
        if node_temp[1][0] == goal_node[0] and node_temp[1][1] == goal_node[1]:      
            print(" Path Found\n")
            node_objects[str(goal_node)] = Node.Node(goal_node,node_temp[0], node)
            break
        
        for next_node, cost in neighbours(node,map):                                    

            if str(next_node) in visited:                                            
                temp = cost + distance[str(node.pos)]                           
                if temp < distance[str(next_node)]:                             
                    distance[str(next_node)] = temp
                    node_objects[str(next_node)].parent = node

            else:                                                                    
                visited.add(str(next_node))
                a_c = cost + distance[str(node.pos)]
                distance[str(next_node)] = a_c
                new_node = Node.Node(next_node, a_c, node_objects[str(node.pos)])
                node_objects[str(next_node)] = new_node
                q.put([a_c, new_node.pos])
    
    reverse_path = []                                                                    
    goal = node_objects[str(goal_node)]                                              
    reverse_path.append(goal_node)                                                       
    parent_node = goal.parent                                                        
    while parent_node:                              
                reverse_path.append(parent_node.pos)                                             
                parent_node = parent_node.parent                                            
            
    path = list(reversed(reverse_path))                                                 

    return node_objects, path

def Animate(node_objects, path, map):
    
    print(" Saving path video...")                                              
    fourcc = VideoWriter_fourcc(*'mp4v')
    video = VideoWriter('./Path_video.mp4', fourcc, float(300), (600, 250)) 
    
    nodes = node_objects.values()                                                   
    nodes = list(nodes)
    img = np.dstack([map.copy() * 0, map.copy() * 0, map.copy() * 255])              
    img = np.uint8(img)
    video.write(img)
    
    
    for i in range(len(nodes)):                                                    
        img[nodes[i].pos[1], nodes[i].pos[0], :] = np.array([255,255,0])
        video.write(img)
        
    for i in range(len(path) - 1):                                                   
        img[path[i][1], path[i][0], :] = np.array([255,0,255])
        video.write(img)
        
    video.release()
    print("Video saved.")
 
    

map = create_map()
goal_node = goal(map)
node_objects, path = Dijkstra_algorithm(goal_node, map)
Animate(node_objects, path, map)



Enter target node(x,y): 240,90
obstacle collision 

Enter target node(x,y): 245,50

 Enter starting node(x,y): 5,5
 Finding path

 Path Found

 Saving path video...
